<a href="https://colab.research.google.com/github/GabrielleRab/SRMPmachine/blob/main/Linear_regression_exoplanets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Linear Regression with Exoplanets** 

We've already explored linear regression in Google Sheets. Now it's time to use our Colab skills to run linear regression analysis using code. We will also see how this method follows the machine learning pipeline.

### **Step 1:** Identify your question


In this case, we will use linear regression to verify that exoplanets follow the laws of physics as observed in our own solar system. We are interested in the correlations between different features of exoplanets and discovering which known features can be used to predict unknown features.


### **Step 2:** Select your data

Let's first read and explore our data. First we need to load in the necessary Python libraries. Run the code below:

In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Import the numpy and pandas package
import numpy as np
import pandas as pd
from scipy import stats

# Data Visualisation
import matplotlib.pyplot as plt 
import seaborn as sns

Next, we will create a dataframe called "exoplanets" with a cleaned version of our exoplanets data (which has had the outlying stars removed):

In [ ]:
exoplanets = pd.DataFrame(pd.read_csv("https://raw.githubusercontent.com/GabrielleRab/SRMPmachine/main/datasets/exoplanets_cleaned.csv"))
exoplanets.head()

### **Step 3:** Choose your method

Our goal here is to understand the relationship between different variables of the exoplanets. Because we expect them to have linear relationships (in some cases after a bit of mathematical transformation), we can use the Linear Regression method. But first, we need to prepare our dataset. 

### **Step 4:** Prepare your data

Even though we are using the cleaned version of our exoplanets dataset, there may be more cleaning necessary becore we can run the linear regression analysis. For example, there may be missing values. 

The code below returns the percent of each column that is made up of blank cells:

In [ ]:
# Calculating percent "null" or blank
exoplanets.isnull().sum()*100/exoplanets.shape[0]


# Discovery year vs radius = disc_year vs pl_radj
# mass of star vs mass of planet = st_mass vs pl_bmassj
# mass of planet vs radius of planet = pl_bmassj vs pl_radj

Let's create a cleaned dataframe whithout any exoplanets that are missing values to avoid introducing false correlations or detract from real correlations:

In [ ]:
# We remove the rows with the NULL values in the dataset and create a cleaned version 
exoplanets_dropna = exoplanets.dropna()

# Double checking that we have removed all null values
exoplanets_dropna.isnull().sum()*100/exoplanets_dropna.shape[0]

Let's see how many exoplanets are left in our cleaned dataset:

In [ ]:
# return the number of rows in the cleaned dataframe
print("exoplanets remaining", len(exoplanets_dropna))

# calculate the number of removed exoplanets
print("exoplanets removed:", len(exoplanets)-len(exoplanets_dropna))

Now that we've handled missing data, we need to check for outliers that might affect our results. Let's look at some box plots for our data:

In [ ]:
# Outlier Analysis
fig, axs = plt.subplots(7, figsize = (6,6))
plt1 = sns.boxplot(exoplanets_dropna['pl_radj'], ax = axs[0])
plt2 = sns.boxplot(exoplanets_dropna['sy_dist'], ax = axs[1])
plt3 = sns.boxplot(exoplanets_dropna['pl_bmassj'], ax = axs[2])
plt4 = sns.boxplot(exoplanets_dropna['pl_orbper'], ax = axs[3])
plt5 = sns.boxplot(exoplanets_dropna['disc_year'], ax = axs[4])
plt6 = sns.boxplot(exoplanets_dropna['st_mass'], ax = axs[5])
plt7 = sns.boxplot(exoplanets_dropna['pl_orbsmax'], ax = axs[6])

plt.tight_layout()

**Bias alert!** Linear regression is very sensitive to outliers, meaning that even with the stars removed, this dataset contains some values that could skew our results. 

You can see how much of an impact the outliers have on our data's distribution by looking at the boxplots without them:

In [ ]:
# Plots without the outliers. 
fig, axs = plt.subplots(7, figsize = (6,6))
plt1 = sns.boxplot(exoplanets_dropna['pl_radj'], ax = axs[0], showfliers = False)
plt2 = sns.boxplot(exoplanets_dropna['sy_dist'], ax = axs[1], showfliers = False)
plt3 = sns.boxplot(exoplanets_dropna['pl_bmassj'], ax = axs[2], showfliers = False)
plt4 = sns.boxplot(exoplanets_dropna['pl_orbper'], ax = axs[3], showfliers = False)
plt5 = sns.boxplot(exoplanets_dropna['disc_year'], ax = axs[4], showfliers = False)
plt6 = sns.boxplot(exoplanets_dropna['st_mass'], ax = axs[5], showfliers = False)
plt7 = sns.boxplot(exoplanets_dropna['pl_orbsmax'], ax = axs[6], showfliers = False)

plt.tight_layout()

There are some considerable outliers in the dataset, so let's remove them and create a cleaned version of our dataframe:

In [ ]:
# remove outliers
exoplanets_clean = exoplanets_dropna.drop(exoplanets_dropna[exoplanets_dropna.pl_radj > 1.8].index)
exoplanets_clean = exoplanets_clean.drop(exoplanets_clean[exoplanets_clean.sy_dist > 2000].index)
exoplanets_clean = exoplanets_clean.drop(exoplanets_clean[exoplanets_clean.pl_bmassj > 0.6].index)
exoplanets_clean = exoplanets_clean.drop(exoplanets_clean[exoplanets_clean.pl_orbper > 90].index)
exoplanets_clean = exoplanets_clean.drop(exoplanets_clean[exoplanets_clean.st_mass > 1.6].index)
exoplanets_clean = exoplanets_clean.drop(exoplanets_clean[exoplanets_clean.pl_orbsmax > 0.5].index)

As is visible from the heatmap, the variable `pl_radj` seems is highly correlated with `pl_bmassj` (+0.88) which is intuitive given that more massive planets are likely to be larger. 

Star radius and planetary mass are also highly correlated (+0.37), which can be explained given that bigger stars form from larger gas clouds which allows more matter for planet formation.

It seems like `disc_year` and `pl_bmassj` also have a high negative correlation (-0.21), because we have been able to observe smaller exoplanets as our technology has improved.

What other correlations do you notice and what do you think might explain them?

Since we are interested in predicting planet redius given the independent variable of mass of the planet, we will assign the independent variable, `pl_bmassj`, in this case, to the variable `X` and the dependent variable, `pl_radj`, to the variable `y`.

In [ ]:
# This has all the relevant relationships. The text above will change according to the features being explored. 

#semi-major axis (pl_irbsmax cubed) vs pl_orbsper squared
X = exoplanets_clean['pl_orbsmax'] ** 3
y = exoplanets_clean['pl_orbper'] ** 2
xlabel = "Orbit Semi-Major Axis cubed"
ylabel = "Orbital period squared"

### **Step 5:** Train the model

Remember how to calculate the slope of a linear graph? <br>
<img src="https://www.katesmathlessons.com/uploads/1/6/1/0/1610286/1163738_orig.png" width="250">


The equation for linear regression is a more complex version of this:<br>
$y = c + m_1x_1 + m_2x_2 + ... + m_nx_n$

-  $y$ is the response
-  $c$ is the intercept
-  $m_1$ is the coefficient for the first feature
-  $m_n$ is the coefficient for the nth feature<br>

In our case:

$y = c + m_1 \times sydist$

The $m$ values are called the model **coefficients** or **model parameters**.

---

We now need to split our variable into training and testing sets. We'll perform this by importing `train_test_split` from the `sklearn.model_selection` library. It is usually a good practice to keep 70% of the data in your train dataset and the rest 30% in your test dataset

In [ ]:
# import train test split function
from sklearn.model_selection import train_test_split

# split the data into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3, random_state = 100)

In [ ]:
# Let's now take a look at the train dataset

X_train.head()

In [ ]:
y_train.head()

You first need to import the `statsmodel.api` library using which you'll perform the linear regression.

In [ ]:
import statsmodels.api as sm

By default, the `statsmodels` library fits a line on the dataset which passes through the origin. But in order to have an intercept, you need to manually use the `add_constant` attribute of `statsmodels`. And once you've added the constant to your `X_train` dataset, you can go ahead and fit a regression line using the `OLS` (Ordinary Least Squares) attribute of `statsmodels` as shown below

In [ ]:
# Add a constant to get an intercept
X_train_sm = sm.add_constant(X_train)

# Fit the resgression line using 'OLS'
lr = sm.OLS(y_train, X_train_sm).fit()

In [ ]:
# Print the parameters, i.e. the intercept and the slope of the regression line fitted
lr.params

In [ ]:
# Performing a summary operation lists out all the different parameters of the regression line fitted
print(lr.summary())
### safi - explain relevant stats, p-values 

####  Looking at some key statistics from the summary

Take a look at the R-squared for this result: 0.928, meaning that 92.8% of the variance in orbital period squared is explained by the orbit semi-major axis cubed.

In scientific studies, the R-squared may need to be above 0.95 for a regression model to be considered reliable, so we're pretty close! In other domains, an R-squared of just 0.3 may be sufficient if there is extreme variability in the dataset.


Our F statistic has a very low p value (0.0002894)
Meaning that the model fit is statistically significant, and the explained variance isn't purely by chance.

---
The fit is significant. Let's visualize how well the model fit the data.

Our linear regression equation was: 

y = mx + c

From the parameters that we get, our linear regression equation becomes:

$ orbital period squared = 0.150294 \times orbit semimajor axis cubed + 55.728726 $


In [ ]:
plt.scatter(X_train, y_train)
plt.title("Training data")
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.plot(X_train, 126954.187818*X_train + 0.365779, 'r')
plt.show()

### **Step 6:** Test the model

Now that we've fitted a regression line on the training dataset, it's time to make some predictions on the test data:

In [ ]:
# Add a constant to X_test
X_test_sm = sm.add_constant(X_test)

# Predict the y values corresponding to X_test_sm
y_pred = lr.predict(X_test_sm)

Let's check the R-squared on the test set. The closer R squared is to 1, the better our correlation:

In [ ]:
from sklearn.metrics import r2_score
r_squared = r2_score(y_test, y_pred)
r_squared

This means that 94.3% of the variance in `pl_radj` in the test set is explained by `pl_bmassj`. This is a great fit! 



Let's visualize the fit on the test set:

In [ ]:
plt.scatter(X_test, y_test)
plt.plot(X_test, 126954.187818*X_test + 0.365779, 'r')
plt.title("Test data")
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.show()

These results are consistent with Kepler's Third Law: the squares of the orbital periods of the planets are directly proportional to the cubes of the semi-major axes of their orbits. Kepler's Third Law implies that the period for a planet to orbit the Sun increases rapidly with the radius of its orbit.

### **Step 7:** Evaluate the model

Let's see if our model can predict the orbital periods of the earth, given its semi-major axis:

In [ ]:
import math
print("Predicted earth orbital period", math.sqrt(126954.187818*1**3 + 0.365779))

Not bad! In reality, it's 365 days.

### **Bonus**: Explore more variables

Below you will see the correlations for every pair of variables in our dataset:

In [ ]:
# Let's see the correlation between different variables.
sns.heatmap(exoplanets_clean.corr(), cmap="YlGnBu", annot = True)
plt.show()

# Discovery year vs radius = disc_year vs pl_radj
# mass of star vs mass of planet = st_mass vs pl_bmassj
# mass of planet vs radius of planet = pl_bmassj vs pl_radj

Choose any two variables you would like to compare and type their names here:

In [ ]:
# make sure you put the variable names in QUOTATION MARKS
x_var = "pl_radj"
y_var = "pl_bmassj"

In [ ]:
# set up data
X_try = exoplanets_clean[x_var]
y_try = exoplanets_clean[y_var]
xlabel = x_var
ylabel = y_var

In [ ]:
# Add a constant to get an intercept
X_try_sm = sm.add_constant(X_try)

# Fit the resgression line using 'OLS'
lr = sm.OLS(y_try, X_try_sm).fit()
lr.params

In [ ]:
# graph your results
plt.scatter(X_try, y_try)
plt.plot(X_try, lr.params[1]*X_try + lr.params[0], 'r')
plt.xlabel(x_var)
plt.ylabel(y_var)
plt.show()